In [ ]:
!pip install anthropic

In [ ]:
!pip install anthropic IPython boto3 botocore

In [124]:
import boto3
import json
import base64
from anthropic import AnthropicBedrock
from IPython.display import Image, JSON
from io import BytesIO
from botocore.config import Config

config = Config(
    region_name = 'us-east-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)
session = boto3.Session() # create a boto3 session to dynamically get and set the region name
bedrock = session.client(service_name='bedrock-runtime', config=config) #creates a Bedrock client

#anthropic_client = AnthropicBedrock(api_key='',  client=client)

#### Função para obter uma string base64 a partir de bytes de imagem

In [125]:
def get_base64_from_bytes(image_bytes):
    resized_io = BytesIO(image_bytes)
    img_str = base64.b64encode(resized_io.getvalue()).decode("utf-8")
    return img_str

#### Calling Function para transcrição de documentos

In [126]:
def transcribe_documents(user_prompt, system_prompt=None, tools_use=None, tool_choice=None, image_bytes=None):
#Prepara o corpo da solicitação - Hiperparâmetros
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4200,
        "temperature": 0,
        "top_k": 250,
        "top_p": 0.999,
        "tools": tools_use,
        "tool_choice": tool_choice,
        #"system": system_prompt,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": get_base64_from_bytes(image_bytes),
                        },
                    },
                    {
                        "type": "text",
                        "text": user_prompt
                    }
                ],
            }
        ],
    }
    if system_prompt is not None:
        body.update({"system": system_prompt})
    
    # Chama o LLM Claude usando um cliente Bedrock
    return bedrock.invoke_model(body=json.dumps(body), modelId = "anthropic.claude-3-haiku-20240307-v1:0")

#### Define e seleciona ferramentas

In [127]:
#tools_use
tools_use = [{
    "name": "transcribe_documents",
    "description": "Extract all <document/> fields with the highest accuracy following <instructions/>",
    "input_schema": {
        "type": "object",
        "properties": {
            "documents": {
                "type": "array",
                "items": {
                    "$ref":"/schemas/document"
                }
            }
        }
    }
}]

#tool_choice
tool_choice = {"type": "tool", "name": "transcribe_documents"}

#### Define Schema JSON

In [131]:
system_prompt = """
<instructions>
  - Noticed that the content is in Portuguese and must be transcribed in that language
  - Ensure to escape quotes in the JSON response
  - Returns "" for missing field values in all rows where missing values
  - Apply dependentSchemas to all <document/> fields
  
</instructions>
<document>
{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "/schemas/document",
    "type": "object",
    "description": "Documento com os campos para transcrição",
    "properties": {
        "tipo_documento": { "properties": { "valor": { "type": "string" } }, "description": "Tipo de documento: Conta de energia" },
        "numero_recibo": { "properties": { "valor": { "type": "string" } }, "description": "Número do recibo ou outro número identificador" },
        "valor_total_documento": { "properties": { "valor": { "type": "number" } }, "description": "Valor total do recibo" },
        "moeda": { "properties": { "valor": { "type": "string" } }, "description": "Tipo de moeda e.g. BRL" },
        "numero_negocio_vendedor": { "properties": { "valor": { "type": "string" } }, "description": "Número de identificação do negócio do vendedor e.g. CNPJ" },
        "nome_vendedor": { "properties": { "valor": { "type": "string" } }, "description": "Nome do negócio que emitiu o recibo" },
        "endereco_vendedor": { "properties": { "valor": { "type": "string" } }, "description": "Endereço do site do vendedor" },
        "telefone_vendedor": { "properties": { "valor": { "type": "string" } }, "description": "Número de telefone do vendedor" },
        "metodo_pagamento": { "properties": { "valor": { "type": "string" } }, "description": "Tipo de pagamento, e.g. EFTPOS, Cartão" },
        "data_emissao": { "properties": { "valor": { "format": "YYYY-MM-DDThh:mm:ss" } }, "description": "Data de emissão do documento" },
        "valor_total_itens_linha": { "properties": { "valor": { "type": "number" } }, "description": "Soma calculada dos campos de valor dos itens de linha" },
        "Tabela de Itens": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "Itens da Fatura": { "type": "string", "description": "Descrição do item de linha" },
                    "Unid.": { "type": "string", "description": "Unidade de medida" },
                    "Quant.": { "type": "number", "description": "Quantidade do item" },
                    "Preço Unit.(R$)": { "type": "number", "description": "Preço unitário do item" },
                    "Valor(R$)": { "type": "number", "description": "Valor do item de linha" },
                    "PIS/COFINS": { "type": "number", "description": "Valor de PIS/COFINS" },
                    "Base Calc. ICMS": { "type": "number", "description": "Base de cálculo do ICMS" },
                    "Aliq. ICMS": { "type": "number", "description": "Alíquota do ICMS" },
                    "ICMS": { "type": "number", "description": "Valor do ICMS" },
                    "Tarifa Unit.": { "type": "number", "description": "Tarifa unitária" }
                }
            }
        },
        "Grandezas Contratadas": {
            "type": "object",
            "description": "Tabela de Grandezas Contratadas",
            "properties": {
                "Demanda Fora Ponta": { "type": "integer", "description": "Valor da demanda fora ponta contratada" }
            }
        },
        "Tabela de Impostos": {
            "type": "object",
            "description": "Tabela de Impostos",
            "properties": {
                "ICMS": {
                    "type": "object",
                    "properties": {
                        "Base de calculo (R$)": { "type": "number", "description": "Base de cálculo do ICMS" },
                        "Aliquota (%)": { "type": "number", "description": "Alíquota do ICMS" },
                        "Valor (R$)": { "type": "number", "description": "Valor do ICMS" }
                    }
                },
                "PASEP": {
                    "type": "object",
                    "properties": {
                        "Base de calculo (R$)": { "type": "number", "description": "Base de cálculo do PASEP" },
                        "Aliquota (%)": { "type": "number", "description": "Alíquota do PASEP" },
                        "Valor (R$)": { "type": "number", "description": "Valor do PASEP" }
                    }
                },
                "COFINS": {
                    "type": "object",
                    "properties": {
                        "Base de calculo (R$)": { "type": "number", "description": "Base de cálculo do COFINS" },
                        "Aliquota (%)": { "type": "number", "description": "Alíquota do COFINS" },
                        "Valor (R$)": { "type": "number", "description": "Valor do COFINS" }
                    }
                }
            }
        }
    },
    "dependentSchemas": {
        "valor": {
            "properties": {
                "inferencia": { "type": "integer", "description": "0=EXPLICITO|1=DERIVADO|2=AUSENTE|3=OUTRO" },
                "fonte": { "type": "string", "description": "Campo fonte apenas para campos explícitos e derivados" }
            }
        }
    }
}
<document/>
"""
regular_prompt = """
Describe documents
"""

In [ ]:
import matplotlib.pyplot as plt

image_path = "/home/docsia/ssd-data/estudos/Claude_calling-function/4e143382dcb3537dbe1c39fb16a04e9d.jpg"

plt.figure(figsize=(12, 14))

plt.imshow(img)
plt.show()

In [143]:
# Leitura da imagem e transcrição do documento
image_bytes = bytearray(open("/home/docsia/ssd-data/estudos/Claude_calling-function/4e143382dcb3537dbe1c39fb16a04e9d.jpg", 'rb').read())  
#image_bytes = bytearray(open("/home/docsia/ssd-data/estudos/calling-function_claude3/2b2d6ddaaf4568c40abf5a44ab87a9b8.jpg", 'rb').read())  

response = transcribe_documents(regular_prompt, system_prompt, tools_use, tool_choice, image_bytes)

response_body = json.loads(response.get('body').read()) 

display(response_body)

{'id': 'msg_bdrk_01CVHDsqCyRgf2WB2YFbMs6s',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-haiku-20240307',
 'stop_sequence': None,
 'usage': {'input_tokens': 3608, 'output_tokens': 2090},
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01EKoahpXABzeFJmEHYEiUi8',
   'name': 'transcribe_documents',
   'input': {'documents': [{'tipo_documento': {'valor': 'Conta de energia'},
      'numero_recibo': {'valor': 'ABR/2024'},
      'valor_total_documento': {'valor': 109760.37},
      'moeda': {'valor': 'BRL'},
      'numero_negocio_vendedor': {'valor': '20.294.0**/****.**'},
      'nome_vendedor': {'valor': 'BIOCOR HOSPITAL DOENÇAS CARDIOVASCULARES'},
      'endereco_vendedor': {'valor': 'AV. BARBACENA, 1.200 - 1º ANDAR - A.L.I - BAIRRO SANTO AGOSTINHO'},
      'telefone_vendedor': {'valor': ''},
      'metodo_pagamento': {'valor': 'TSEE FOI CRIADA PELA LEI Nº 10.438, DE 26 DE ABRIL DE 2002'},
      'data_emissao': {'valor': '2024-05-13T00:00:00'},
      'valor_total_iten

In [133]:
display(JSON(response_body['content'][0]['input'], root='content'))

<IPython.core.display.JSON object>

#### Classificação de documentos

In [112]:
tools_use = [{
    "name": "classify_documents",
    "description": "Perform document classification",
    "input_schema": {
        "type": "object",
        "properties": {
            "documents": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "type": { "description": "Type of document", "enum": ["recibo", "invoice", "conta de energia"] },
                        "country_code": { "description": "Country code e.g. AU/US/UK" },
                        "document_number": { "description": "The document number or other identifier number" }

                    }
                }
            }
        }
    }
}]
tool_choice = {"type": "tool", "name": "classify_documents"}

regular_prompt = """
Classify documents
"""

In [134]:
# Leitura da imagem e classificação dos documentos
image_bytes = bytearray(open("/home/docsia/ssd-data/estudos/Claude_calling-function/4e143382dcb3537dbe1c39fb16a04e9d.jpg", 'rb').read())  
response = transcribe_documents(regular_prompt, None, tools_use, tool_choice, image_bytes)

# Processamento da resposta
response_body = json.loads(response.get('body').read()) 

# Exibição da resposta
display(response_body)

{'id': 'msg_bdrk_012Ft6kzayUKcDh12T2sa8ew',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-haiku-20240307',
 'stop_sequence': None,
 'usage': {'input_tokens': 1997, 'output_tokens': 61},
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_018bKJSGWvpeUYy3aDeNvVGu',
   'name': 'transcribe_documents',
   'input': {'documents': [{'type': 'electricity_bill',
      'accuracy': 'high'}]}}],
 'stop_reason': 'tool_use'}

In [135]:
display(JSON(response_body['content'][0]['input'], root='content'))

<IPython.core.display.JSON object>